In [1]:
import numpy as np 
import pandas as pd 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import matplotlib.pyplot as plt
from torchvision import transforms, utils
import torchvision
from torchvision import models
from functools import partial
import csv
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
from PIL import Image

In [2]:
torch.__version__

'1.13.1+cu116'

In [3]:
df =  pd.read_csv('D:/Leaning in Glasgow/Msc Project/DATA/FERPlus-master/data/FER2013Train/label.csv')
df

,Image name,size,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,fer0000000.png,"(0, 0, 48, 48)",4,0,0,1,3,2,0,0,0,0
1,fer0000001.png,"(0, 0, 48, 48)",6,0,1,1,0,0,0,0,2,0
2,fer0000002.png,"(0, 0, 48, 48)",5,0,0,3,1,0,0,0,1,0
3,fer0000003.png,"(0, 0, 48, 48)",4,0,0,4,1,0,0,0,1,0
4,fer0000004.png,"(0, 0, 48, 48)",9,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
28553,fer0028633.png,"(0, 0, 48, 48)",1,0,2,0,0,0,7,0,0,0
28554,fer0028634.png,"(0, 0, 48, 48)",0,0,1,0,9,0,0,0,0,0
28555,fer0028635.png,"(0, 0, 48, 48)",0,0,1,9,0,0,0,0,0,0
28556,fer0028636.png,"(0, 0, 48, 48)",5,0,0,1,0,1,0,0,3,0


In [4]:
root_dir='D:/Leaning in Glasgow/Msc Project/DATA/extra/FER2013Train'

In [5]:
import cv2

In [7]:
def save_to_csv(data, file_path):
    headers = ["Image name", "Label"]
    
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)

In [10]:
import imgaug as ia
from imgaug import augmenters as iaa

headers = ["Image name", "Label"]
augmented_5_info = []
augmented_6_info = []
augmented_7_info = []


for idx in range(len(df)):
    img_name = os.path.join(root_dir, df.iloc[idx, 0])
    label_dist =  np.array(df.iloc[idx , 2:-2].values.tolist())
    label_dist = np.divide(label_dist, 10)
    label = np.argmax(label_dist)

    image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
    
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),             # Horizontal flip
        iaa.Affine(rotate=(-25, 25)), # Rotation
        iaa.GaussianBlur(sigma=(0, 1.0)),  # Gaussian blur
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Gaussian noise
        iaa.Multiply((0.8, 1.2))    # Brightness multiplier
    ])
    
    file_name = df.iloc[idx, 0]
    output_path = 'D:/Leaning in Glasgow/Msc Project/DATA/extra/' 
    csv_file_path_5 = "D:/Leaning in Glasgow/Msc Project/DATA/extra/label5/label5.csv"
    csv_file_path_6 = "D:/Leaning in Glasgow/Msc Project/DATA/extra/label6/label6.csv"
    csv_file_path_7 = "D:/Leaning in Glasgow/Msc Project/DATA/extra/label7/label7.csv"
    
    if label == 5:
        for idx in range(10):
            augmented_filename = 'aug'+str(idx+1)+'_'+file_name
            augmented_image = seq.augment_image(image)
            
            augmented_path = output_path+ '/label5/' + augmented_filename
            cv2.imwrite(augmented_path, augmented_image)
            
            augmented_5_info.append([augmented_filename, 5])
    
    if label == 6:
        for idx in range(5):
            augmented_filename = 'aug'+str(idx+1)+'_'+file_name
            augmented_image = seq.augment_image(image)
            
            augmented_path = output_path+ '/label6/' + augmented_filename
            cv2.imwrite(augmented_path, augmented_image)
            
            augmented_6_info.append([augmented_filename, 6])
    
    if label == 7:
        for idx in range(10):
            augmented_filename = 'aug'+str(idx+1)+'_'+file_name
            augmented_image = seq.augment_image(image)
            
            augmented_path = output_path+ '/label7/' + augmented_filename
            cv2.imwrite(augmented_path, augmented_image)
            
            augmented_7_info.append([augmented_filename, 7])
        
        
    save_to_csv(augmented_5_info, csv_file_path_5)
    save_to_csv(augmented_6_info, csv_file_path_6)
    save_to_csv(augmented_7_info, csv_file_path_7)

        
        
        

In [9]:
augmented_5_info

[['aug1_fer0000298.png', 5],
 ['aug2_fer0000298.png', 5],
 ['aug3_fer0000298.png', 5],
 ['aug4_fer0000298.png', 5],
 ['aug5_fer0000298.png', 5],
 ['aug6_fer0000298.png', 5],
 ['aug7_fer0000298.png', 5],
 ['aug8_fer0000298.png', 5],
 ['aug9_fer0000298.png', 5],
 ['aug10_fer0000298.png', 5],
 ['aug1_fer0000480.png', 5],
 ['aug2_fer0000480.png', 5],
 ['aug3_fer0000480.png', 5],
 ['aug4_fer0000480.png', 5],
 ['aug5_fer0000480.png', 5],
 ['aug6_fer0000480.png', 5],
 ['aug7_fer0000480.png', 5],
 ['aug8_fer0000480.png', 5],
 ['aug9_fer0000480.png', 5],
 ['aug10_fer0000480.png', 5],
 ['aug1_fer0001251.png', 5],
 ['aug2_fer0001251.png', 5],
 ['aug3_fer0001251.png', 5],
 ['aug4_fer0001251.png', 5],
 ['aug5_fer0001251.png', 5],
 ['aug6_fer0001251.png', 5],
 ['aug7_fer0001251.png', 5],
 ['aug8_fer0001251.png', 5],
 ['aug9_fer0001251.png', 5],
 ['aug10_fer0001251.png', 5],
 ['aug1_fer0001540.png', 5],
 ['aug2_fer0001540.png', 5],
 ['aug3_fer0001540.png', 5],
 ['aug4_fer0001540.png', 5],
 ['aug5_fer

In [44]:
for idx in range(3):
    print(idx)

0
1
2
